In [3]:
import os
import cv2
import numpy as np
from PIL import Image
import pathlib
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img12

In [2]:
imgs_path = 'images'
masks_path = 'masks'
n_img_path = 'n_images'
n_masks_path = 'n_mask'

In [11]:
images_names_re = os.listdir(imgs_path)

In [ ]:
for filename in images_names_re:
    print(filename)
    img = Image.open(imgs_path+'/'+filename)
    img = img.resize((640,640))
    img.save(n_img_path+'/'+filename)

In [13]:
mask_names_re = os.listdir(masks_path)

In [ ]:
for filename in mask_names_re:
    print(filename)
    img = Image.open(masks_path+'/'+filename)
    img = img.resize((640,640))
    img.save(n_masks_path+'/'+filename)

In [5]:
def segmentImg(img , mask):
    image = cv2.imread(img)
    mask = cv2.imread(mask)
#     to convert mask to gray scale
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
# to find the shape of the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#  to get the largest shape from the mask
    largest_contour = max(contours, key=cv2.contourArea)
#  to return the shape in four point to cropped the image and the mask
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Normalize the bounding box coordinates
    width = image.shape[1]
    height = image.shape[0]
    # print(width , height)
    x /= width
    y /= height
    w /= width
    h /= height
    
    return (x) , (y) , (w) , (h) 

In [12]:
x, y, w, h = segmentImg('./n_images/ISIC_0024310.jpg','./n_mask/ISIC_0024310_segmentation.png')
print('x : ', x  , '\ny : ' , y  , '\nw : ', w  , '\nh : ' , h )

x *= 640
y *= 640
w *= 640
h *= 640

w += x
h+= y

img = cv2.imread('./n_images/ISIC_0024310.jpg')
cv2.rectangle(img ,(int(x) , int(y)) , (int(w) , int(h)) , (0,255,255) ,2)
cv2.imshow("img : " , img)
cv2.waitKey(0)
cv2.destroyAllWindows()

x :  0.11875 
y :  0.171875 
w :  0.665625 
h :  0.60625


## that we used to create the bounding box based on the yolo model label file

In [1]:
def segmentImg11(img, mask):
    image = cv2.imread(img)
    mask = cv2.imread(mask)

    # Convert mask to grayscale
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # Find the contours of the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    # print(largest_contour)
    # Check if the contour is closed and large enough
    # Get the minimum area rectangle that is rotated to align with the principal axis of the object
    rect = cv2.minAreaRect(largest_contour)
    print(rect)
    # Get the coordinates of the rotated rectangle
    (x, y), (w, h), angle = rect

    # Convert the rotated rectangle coordinates to a bounding box
    # Convert the rotated rectangle coordinates to an ellipse
    # ellipse = cv2.fitEllipse(largest_contour)

    # # Draw the ellipse on the image
    # cv2.ellipse(image, ellipse, (0, 255, 0), 2)

    # Normalize the bounding box coordinates
    width = image.shape[1]
    height = image.shape[0]
    # print(width , height)
    x /= width
    y /= height
    w /= width
    h /= height

    # Return the bounding box coordinates without normalization
    return x, y, w, h
    # Return the bounding box coordinates without normalization
    # return box


In [14]:
x, y, w, h = segmentImg11('./n_images/ISIC_0024310.jpg','./n_mask/ISIC_0024310_segmentation.png')
print('x : ', x  , '\ny : ' , y  , '\nw : ', w  , '\nh : ' , h )

((276.0614929199219, 287.85589599609375), (410.7248840332031, 373.43975830078125), 16.76255226135254)
x :  0.43134608268737795 
y :  0.44977483749389646 
w :  0.6417576313018799 
h :  0.5834996223449707


In [15]:
image = cv2.imread('./n_images/ISIC_0024310.jpg')

In [17]:
x *= 640
y *= 640
w *= 640
h *= 640

w += x
h+= y
cv2.rectangle(image, (int(x), int(y)), (int(w), int(h)), (0, 255, 0), 3)  # Green rectangle
cv2.imshow("Image with Bounding Box", image)
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()


In [72]:
images_path = 'n_images'
masks_path = 'n_mask'
processed_path = 'labels'

In [144]:
truth = pd.read_csv('GroundTruth.csv')
file_name = truth.iloc[:,0]
MEL = truth.iloc[:,1]
NV = truth.iloc[:,2]
BCC = truth.iloc[:,3]
AKIEC = truth.iloc[:,4]
BKL = truth.iloc[:,5]
DF = truth.iloc[:,6]
VASC = truth.iloc[:,7]

In [145]:
name_len = len(file_name[0])

In [75]:
for i , j in enumerate(file_name):
    for filename in os.listdir(images_path):
        if j == filename[:name_len] and MEL[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(4) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
        if j == filename[:name_len] and NV[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(5) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                
        if j == filename[:name_len] and BCC[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(1) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                
        elif j == filename[:name_len] and AKIEC[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(0) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                
        if j == filename[:name_len] and BKL[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(2) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                
        if j == filename[:name_len] and DF[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(3) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                
        if j == filename[:name_len] and VASC[i] == 1:
            xmin, ymin, xmax, ymax = segmentImg11(images_path + '/' + filename , masks_path+'/'+j+'_segmentation.png' )
            with open('labels/' +filename[:name_len] +'.txt' , "a+") as f:
                f.write(str(6) + ' ' +str(xmin)+' ' +str(ymin)+' ' +str(xmax)+' '+str(ymax))
                

((367.5094299316406, 302.37890625), (349.1114501953125, 586.94775390625), 0.8680514693260193)
((356.4034423828125, 322.92852783203125), (538.3344116210938, 414.6428527832031), 79.545166015625)
((331.3099365234375, 358.89691162109375), (672.8701171875, 336.8175048828125), 60.09406280517578)
((266.55731201171875, 396.63763427734375), (478.1966857910156, 386.14434814453125), 87.82868957519531)
((276.0614929199219, 287.85589599609375), (410.7248840332031, 373.43975830078125), 16.76255226135254)
((314.8091735839844, 375.45196533203125), (330.71636962890625, 520.44482421875), 46.517417907714844)
((323.91302490234375, 321.5640869140625), (550.2301025390625, 418.4742126464844), 87.11485290527344)
((302.60235595703125, 317.76715087890625), (312.11614990234375, 196.74000549316406), 2.361374616622925)
((318.33056640625, 324.2716369628906), (260.7353515625, 243.200439453125), 0.993468701839447)
((298.7767333984375, 293.9547424316406), (215.17303466796875, 241.83709716796875), 2.806370258331299)
((

In [147]:
import shutil
import random


In [148]:
m_count = 0
NV_count = 0 
BCC_count = 0
AK_count = 0
BKL_count = 0
DF_count = 0
Va_count = 0

In [149]:
train_images = 'train1/images'
test_images = 'test1/images'
valid_images = 'valid1/images'

train_lb = 'train1/labels'
test_lb = 'test1/labels'
valid_lb = 'valid1/labels'

images_dire = 'n_images'
label_dire = 'labels'
labels_cc = os.listdir(label_dire)
images_cc = os.listdir(images_dire)


In [150]:
labels_cc.sort()
len(labels_cc)

10015

In [151]:
images_cc.sort()

In [152]:
def cop_img(m_count , filename , im_path ):
    if m_count <= 200:
        shutil.copyfile(os.path.join(im_path, filename), os.path.join(train_images, filename))
    if m_count > 200 and m_count <=280:
        shutil.copyfile(os.path.join(im_path, filename), os.path.join(test_images, filename))
    if m_count > 280 and m_count <=300:
        shutil.copyfile(os.path.join(im_path, filename), os.path.join(valid_images, filename))
    else:
        print('done')

In [153]:
def cop_lab(m_count  , lb_name , lb_path ):
    if m_count <= 200:
        shutil.copyfile(os.path.join(lb_path, lb_name), os.path.join(train_lb, lb_name))
    if m_count > 200 and m_count <=280:
        shutil.copyfile(os.path.join(lb_path, lb_name), os.path.join(test_lb, lb_name))      
    if m_count > 280 and m_count <=300:
        shutil.copyfile(os.path.join(lb_path, lb_name), os.path.join(valid_lb, lb_name))
    else:
        print('done')

In [157]:
for i , j in enumerate(file_name):
    for filename in images_cc:
        if j == filename[:name_len] and MEL[i] == 1:
            m_count +=1    
            cop_img(m_count , filename , images_dire)

        if j == filename[:name_len] and NV[i] == 1:
            NV_count +=1               
            cop_img(NV_count , filename  , images_dire )
                
        if j == filename[:name_len] and BCC[i] == 1:
            BCC_count +=1
            cop_img(BCC_count , filename  , images_dire )
                
        if j == filename[:name_len] and AKIEC[i] == 1:
            AK_count +=1
            cop_img(AK_count , filename  , images_dire )
                
        if j == filename[:name_len] and BKL[i] == 1:
            BKL_count +=1
            cop_img(BKL_count , filename  , images_dire )
                
        if j == filename[:name_len] and DF[i] == 1:
            DF_count +=1
            cop_img(DF_count , filename  , images_dire )
                
        if j == filename[:name_len] and VASC[i] == 1:
            Va_count +=1
            cop_img(Va_count , filename  , images_dire )


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [158]:
print(m_count , AK_count , BKL_count , NV_count , BCC_count , DF_count , Va_count)

1113 327 1099 6707 514 115 142


In [159]:
m_count , AK_count , BKL_count , NV_count , BCC_count , DF_count , Va_count = 0,0,0,0,0,0,0

In [160]:
for i , j in enumerate(file_name):
    for filename in labels_cc:
        if j == filename[:name_len] and MEL[i] == 1:
            m_count +=1
            
            cop_lab(m_count , filename , label_dire)

        if j == filename[:name_len] and NV[i] == 1:
            NV_count +=1               
            cop_lab(NV_count , filename  , label_dire )
                
        if j == filename[:name_len] and BCC[i] == 1:
            BCC_count +=1
            cop_lab(BCC_count , filename  , label_dire )
                
        if j == filename[:name_len] and AKIEC[i] == 1:
            AK_count +=1
            cop_lab(AK_count , filename  , label_dire )
                
        if j == filename[:name_len] and BKL[i] == 1:
            BKL_count +=1
            cop_lab(BKL_count , filename  , label_dire )
                
        if j == filename[:name_len] and DF[i] == 1:
            DF_count +=1
            cop_lab(DF_count , filename  , label_dire )
                
        if j == filename[:name_len] and VASC[i] == 1:
            Va_count +=1
            cop_lab(Va_count , filename  , label_dire )


done
done
done
done
done
done


done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [161]:
print(m_count , AK_count , BKL_count , NV_count , BCC_count , DF_count , Va_count)

1113 327 1099 6705 514 115 142


In [77]:
source_images = 'n_images'

In [78]:
# Define the destination directories for the train, test, and validation sets
train_dir = "train/images"
test_dir = "test/images"
valid_dir = "valid/images"

In [79]:
# Create the destination directories if they don't already exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

In [80]:
# Get a list of all the images in the source directory
images = os.listdir(source_images)

In [81]:
num_images = len(images)
num_train = int(num_images * 0.8)
num_test = int(num_images * 0.1)
num_valid = num_images - num_train - num_test

In [82]:
# Copy the selected images to the respective destination directories
for i, image in enumerate(images):
    if i < num_train:
        shutil.copyfile(os.path.join(source_images, image), os.path.join(train_dir, image))
    elif i < num_train + num_test:
        shutil.copyfile(os.path.join(source_images, image), os.path.join(test_dir, image))
    else:
        shutil.copyfile(os.path.join(source_images, image), os.path.join(valid_dir, image))

In [83]:
# Print the number of images in each directory
print("Number of images in train directory:", len(os.listdir(train_dir)))
print("Number of images in test directory:", len(os.listdir(test_dir)))
print("Number of images in valid directory:", len(os.listdir(valid_dir)))

Number of images in train directory: 8012
Number of images in test directory: 1001
Number of images in valid directory: 1002


In [84]:
source_labels = 'labels'

In [85]:
# Define the destination directories for the train, test, and validation sets
train_labels = "train/labels"
test_labels = "test/labels"
valid_labels = "valid/labels"

In [86]:
# Create the destination directories if they don't already exist
os.makedirs(train_labels, exist_ok=True)
os.makedirs(test_labels, exist_ok=True)
os.makedirs(valid_labels, exist_ok=True)

In [87]:
labels = os.listdir(source_labels)

In [88]:
num_labels = len(labels)
num_train = int(num_labels * 0.8)
num_test = int(num_labels * 0.1)
num_valid = num_labels - num_train - num_test

In [89]:
# Copy the selected images to the respective destination directories
for i, image in enumerate(labels):
    if i < num_train:
        shutil.copyfile(os.path.join(source_labels, image), os.path.join(train_labels, image))
    elif i < num_train + num_test:
        shutil.copyfile(os.path.join(source_labels, image), os.path.join(test_labels, image))
    else:
        shutil.copyfile(os.path.join(source_labels, image), os.path.join(valid_labels, image))

In [162]:
# Print the number of images in each directory
print("Number of images in train directory:", len(os.listdir(train_labels)))
print("Number of images in test directory:", len(os.listdir(test_labels)))
print("Number of images in valid directory:", len(os.listdir(valid_labels)))

Number of images in train directory: 8012
Number of images in test directory: 1001
Number of images in valid directory: 1002
